In [6]:
# from spacy import load, prefer_gpu 
from helpers import get_dates_from_file
from glob import glob
from tqdm import tqdm
from datetime import datetime
import pandas as pd 
from spacy import load, prefer_gpu, displacy
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
# !pip install https://huggingface.co/csae8092/de_MRP_NER/resolve/main/de_MRP_NER-any-py3-none-any.whl --user
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
prefer_gpu()

# Parameters
useSpacy = True
folder = "./results/NYT"
language = "en"
model_dict = {
    "fr" : pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple", device = 0),
    #"de" : load("de_RTA_NER"),
    "en" : load("en_core_web_trf")
}

nlp = model_dict[language]



In [ ]:
# folder = "./results/Figaro"
year = 1920
def sample_year(folder: str, year: int, lt: int, rt: float, nb_sample = 200):
    path = folder  + "/" + str(year)
    results = []
    for file in tqdm(glob(path + "/*/*.csv")):        # Compute the nb of dates
        results.append(pd.read_csv(file))
    df = pd.concat(results)
    df = df[(df.Length > lt) & (df.Ratio > rt)]
    sampled_df = df.sample(n = nb_sample)
    return sampled_df

to_anotate = sample_year(folder, year,100,0.8)
to_anotate.to_csv("to_annotate.csv")
to_anotate = pd.read_csv("to_annotate.csv")

def annotation_generator(df: pd.DataFrame, pipeline):
    for i, elem in enumerate(df.Text):
        doc = pipeline(elem)
        print(doc)
        displacy.serve(doc, style="ent")
        yield i


In [8]:
def annotation_generator(df: pd.DataFrame, pipeline):
    for i, elem in enumerate(df.Text[30:]):
        doc = pipeline(elem)
        print(i)
        displacy.serve(doc, style="ent")
        yield i
generator = annotation_generator(to_anotate,nlp)

In [11]:
next(generator)

2



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


2

In [45]:
to_anotate.Text.to_csv('filepath.txt', sep=' ', index=False)
def fr_annotation_generator(df):
    for i, elem in enumerate(df.Text):
        doc = nlp(elem)
        print([d["word"] for d in doc if d["entity_group"] =="DATE"])
        yield i
generator = fr_annotation_generator(to_anotate)

In [71]:
next(generator)

['hier matin']


25